In [73]:
from Crypto.PublicKey import RSA
import numpy as np
import base64
from binascii import hexlify,unhexlify

In [4]:
key = RSA.import_key(open("pubkey.pem").read())
key

RsaKey(n=188198812920607963838697239461650439807163563379417382700763356422988859715234665485319060606504743045317388011303396716199692321205734031879550656996221305168759307650257059, e=65537)

http://factordb.com/index.php?query=188198812920607963838697239461650439807163563379417382700763356422988859715234665485319060606504743045317388011303396716199692321205734031879550656996221305168759307650257059

In [10]:
p = 398075086424064937397125500550386491199064362342526708406385189575946388957261768583317
q = 472772146107435302536223071973048224632914695302097116459852171130520711256363590397527

In [41]:
p*q == key.n

True

In [72]:
cyphertext = "e8oQDihsmkvjT3sZe+EE8lwNvBEsFegYF6+OOFOiR6gMtMZxxba/bIgLUD8pV3yEf0gOOfHuB5bC3vQmo7bE4PcIKfpFGZBA"

In [1]:
# extended greater common divisor
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


# modular inverse
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

    
# ciphertext to integer
def ct2int(ct):
    return int(hexlify(base64.b64decode(ct)), 16)


# integer to plaintext
def int2pt(m):
    hexm = hex(m)[2:]  # .strip("L")
    if len(hexm) % 2:
        hexm = '0' + hexm
    return unhexlify(hexm)

In [86]:
c = ct2int(cyphertext)
c

119597105930544804939860850116469838181273811698209889318158329611052747100532130142039909784725021654968490445938734726928723104382945308422050675301929768086395211461791808

In [76]:
lambda_of_n = np.lcm(p-1, q-1)
lambda_of_n

94099406460303981919348619730825219903581781689708691350381678211494429857617332742659094879636105772538727331365436640741930171074044704027342209817757419034272841145638108

In [79]:
print(f"gcd(e, λ(n)) = {np.gcd(key.e, lambda_of_n)} and modinv(e, λ(n)) = {d*key.e % lambda_of_n}")

gcd(e, λ(n)) = 1 and modinv(e, λ(n)) = 1


In [66]:
d = modinv(key.e, lambda_of_n)
d

48318251158920145864930035723053089097690375168562443830554407970661009102774592695057202204701378327623682075089028698815133956650193819978456750305738325912711134934968241

In [81]:
m = pow(c, d, key.n)
m

7866435180719351620950140112257156212489132106650279078539796875908190600648028413346440808778749178067889240565288397210143595961732257042711322381634908920272905584906

In [89]:
int2pt(m)

b'\x02\x15\x9b\x07@\x1a]\xfe\x93&\xa7\xa8\xdcTq\xed\x8d?\x93\xa6V\x16\xdb\t\x16\xae\xcf\x132U\x02T\xfe\xd5Ae"h\x9eGk\x02\x14\x1d\xee`R!\x86\xe0`e\xff\xd8\xf0\x00up2l6DnaIhZgxA\n'